In [1]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np

#noise function
def make_noisy(image):
    noisy = image + 0.2 * np.random.rand(10, 5)
    noisy = noisy / noisy.max()
    return noisy

#bayt changing function
def make_random_pixel(image):
    rand_index = np.random.randint(0, 50)
    rand_image = np.array(image)
    rand_image[int(rand_index / 5), int(rand_index % 5)] = np.where(rand_image[int(rand_index / 5), int(rand_index % 5)] == 1.0, 0.0, 1.0)
    return rand_image

#activation function
def sigmoid(x):

    return 1/(1 + np.exp(-x))

#derivation of activation function
def der_sigmoid(x):
    return np.exp(x)/((np.exp(x)+1)**2)


image_0 = [[0, 0, 0, 0, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 1, 1, 1, 0],
           [0, 0, 0, 0, 0]]
image_2 = [[0, 0, 0, 0, 0],
           [1, 1, 1, 1, 0],
           [1, 1, 1, 1, 0],
           [1, 1, 1, 1, 0],
           [0, 0, 0, 0, 0],
           [0, 1, 1, 1, 1],
           [0, 1, 1, 1, 1],
           [0, 1, 1, 1, 1],
           [0, 1, 1, 1, 1],
           [0, 0, 0, 0, 0]]
image_4 = [[0,1,1,1,1],
           [0,1,1,1,1],
           [0,1,1,1,1],
           [0,1,1,1,1],
           [0,1,1,1,1],
           [0,1,0,1,1],
           [0,1,0,1,1],
           [0,0,0,0,0],
           [1,1,0,1,1],
           [1,1,0,1,1]]
image_7 = [[0, 0, 0, 0, 1],
           [1, 1, 1, 0, 1],
           [1, 1, 1, 0, 1],
           [1, 1, 1, 0, 1],
           [1, 1, 0, 0, 0],
           [1, 1, 1, 0, 1],
           [1, 1, 1, 0, 1],
           [1, 1, 1, 0, 1],
           [1, 1, 1, 0, 1],
           [1, 1, 1, 0, 1]]


zeros = [[image_0, [1, 0, 0, 0]],
        [make_noisy(image_0), [1, 0, 0, 0]],
        [make_noisy(image_0), [1, 0, 0, 0]],
        [make_noisy(image_0), [1, 0, 0, 0]],
        [make_random_pixel(image_0), [1, 0, 0, 0]],
        [make_random_pixel(image_0), [1, 0, 0, 0]],
        [make_random_pixel(image_0), [1, 0, 0, 0]],
        [make_noisy(image_0), [1, 0, 0, 0]],
        [make_noisy(image_0), [1, 0, 0, 0]],
        [make_noisy(image_0), [1, 0, 0, 0]],
        [make_random_pixel(image_0), [1, 0, 0, 0]],
        [make_random_pixel(image_0), [1, 0, 0, 0]],
        [make_random_pixel(image_0), [1, 0, 0, 0]]]
twos = [[image_2, [0, 1, 0, 0]],
        [make_noisy(image_2), [0, 1, 0, 0]],
        [make_noisy(image_2), [0, 1, 0, 0]],
        [make_noisy(image_2), [0, 1, 0, 0]],
        [make_random_pixel(image_2), [0, 1, 0, 0]],
        [make_random_pixel(image_2), [0, 1, 0, 0]],
        [make_random_pixel(image_2), [0, 1, 0, 0]],
        [make_noisy(image_2), [0, 1, 0, 0]],
        [make_noisy(image_2), [0, 1, 0, 0]],
        [make_noisy(image_2), [0, 1, 0, 0]],
        [make_random_pixel(image_2), [0, 1, 0, 0]],
        [make_random_pixel(image_2), [0, 1, 0, 0]],
        [make_random_pixel(image_2), [0, 1, 0, 0]]]
fours = [[image_4, [0, 0, 1, 0]],
        [make_noisy(image_4), [0, 0, 1, 0]],
        [make_noisy(image_4), [0, 0, 1, 0]],
        [make_noisy(image_4), [0, 0, 1, 0]],
        [make_random_pixel(image_4), [0, 0, 1, 0]],
        [make_random_pixel(image_4), [0, 0, 1, 0]],
        [make_random_pixel(image_4), [0, 0, 1, 0]],
        [make_noisy(image_4), [0, 0, 1, 0]],
        [make_noisy(image_4), [0, 0, 1, 0]],
        [make_noisy(image_4), [0, 0, 1, 0]],
        [make_random_pixel(image_4), [0, 0, 1, 0]],
        [make_random_pixel(image_4), [0, 0, 1, 0]],
        [make_random_pixel(image_4), [0, 0, 1, 0]]]
sevens = [[image_7, [0, 0, 0, 1]],
        [make_noisy(image_7), [0, 0, 0, 1]],
        [make_noisy(image_7), [0, 0, 0, 1]],
        [make_noisy(image_7), [0, 0, 0, 1]],
        [make_noisy(image_7), [0, 0, 0, 1]],
        [make_noisy(image_7), [0, 0, 0, 1]],
        [make_noisy(image_7), [0, 0, 0, 1]],
        [make_random_pixel(image_7), [0, 0, 0, 1]],
        [make_random_pixel(image_7), [0, 0, 0, 1]],
        [make_random_pixel(image_7), [0, 0, 0, 1]],
        [make_random_pixel(image_7), [0, 0, 0, 1]],
        [make_random_pixel(image_7), [0, 0, 0, 1]],
        [make_random_pixel(image_7), [0, 0, 0, 1]]]

random.shuffle(zeros)
random.shuffle(twos)
random.shuffle(fours)
random.shuffle(sevens)

train_datas = np.concatenate((np.concatenate((zeros[:9], twos[:9])), np.concatenate((fours[:9], sevens[:9]))))
test_datas = np.concatenate((np.concatenate((zeros[9:], twos[9:])), np.concatenate((fours[9:], sevens[9:]))))

#generating train and test set
train_data = []
for i in range(len(train_datas)):
    if train_datas[i][1] == [1, 0, 0, 0]:
        train_data.append(np.append(train_datas[i][0], [0]))
    elif train_datas[i][1] == [0, 1, 0, 0]:
        train_data.append(np.append(train_datas[i][0], [1]))
    elif train_datas[i][1] == [0, 0, 1, 0]:
        train_data.append(np.append(train_datas[i][0], [2]))
    elif train_datas[i][1] == [0, 0, 0, 1]:
        train_data.append(np.append(train_datas[i][0], [3]))

test_data = []

for i in range(len(test_datas)):
    if test_datas[i][1] == [1, 0, 0, 0]:
        test_data.append(np.append(test_datas[i][0], [0]))
    elif test_datas[i][1] == [0, 1, 0, 0]:
        test_data.append(np.append(test_datas[i][0], [1]))
    elif test_datas[i][1] == [0, 0, 1, 0]:
        test_data.append(np.append(test_datas[i][0], [2]))
    elif test_datas[i][1] == [0, 0, 0, 1]:
        test_data.append(np.append(test_datas[i][0], [3]))


train_df = pd.DataFrame(train_data)

X_train = train_df.drop(50, axis=1).values
y_train = train_df[50].values

test_df = pd.DataFrame(test_data)

X_test = test_df.drop(50, axis=1).values
y_test = test_df[50].values


X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

X_train = torch.from_numpy(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=50, out_features=25)
        self.fc2 = nn.Linear(in_features=25, out_features=10)
        self.output = nn.Linear(in_features=10, out_features=4)


    def forward(self, x):
        x = F.sigmoid(self.fc1(x))  
        x = F.sigmoid(self.fc2(x))
        x = self.output(x)
        return x


model = ANN()

criterion = nn.CrossEntropyLoss()       #hata fonksiyonu
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01) #optimizasyon türü

#eğitim
epochs = 1000
loss_arr = []
for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    loss_arr.append(loss)

    if loss < 0.0001:
        break

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(f'Loss: {loss}')
#test
predict_out = model(X_test)
_, predict_y = torch.max(predict_out, 1)

print('prediction accuracy', accuracy_score(y_test.data, predict_y.data))


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\oguzh\anaconda3\envs\mypython38\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Loss: 0.07819008827209473
prediction accuracy 1.0
